In [6]:
# imported dependencies
import pennylane as qml
from itertools import product
import numpy as np
import random
qml.version()

'0.29.1'

In [3]:
from pennylane.pauli import pauli_group
n_qubits = 3
t = 0 
for p in pauli_group(n_qubits):
    t += 1
    matrix = qml.matrix(p)
    identityMatrix = -np.identity(matrix.shape[0])
    # isNegative = np.allclose(matrix,identityMatrix)
    # if isNegative:
    #     print("wow")
    #print(qml.matrix(p))
    print(p)
print(t)


Identity(wires=[0])
PauliZ(wires=[2])
PauliZ(wires=[1])
PauliZ(wires=[1]) @ PauliZ(wires=[2])
PauliZ(wires=[0])
PauliZ(wires=[0]) @ PauliZ(wires=[2])
PauliZ(wires=[0]) @ PauliZ(wires=[1])
PauliZ(wires=[0]) @ PauliZ(wires=[1]) @ PauliZ(wires=[2])
PauliX(wires=[2])
PauliY(wires=[2])
PauliZ(wires=[1]) @ PauliX(wires=[2])
PauliZ(wires=[1]) @ PauliY(wires=[2])
PauliZ(wires=[0]) @ PauliX(wires=[2])
PauliZ(wires=[0]) @ PauliY(wires=[2])
PauliZ(wires=[0]) @ PauliZ(wires=[1]) @ PauliX(wires=[2])
PauliZ(wires=[0]) @ PauliZ(wires=[1]) @ PauliY(wires=[2])
PauliX(wires=[1])
PauliX(wires=[1]) @ PauliZ(wires=[2])
PauliY(wires=[1])
PauliY(wires=[1]) @ PauliZ(wires=[2])
PauliZ(wires=[0]) @ PauliX(wires=[1])
PauliZ(wires=[0]) @ PauliX(wires=[1]) @ PauliZ(wires=[2])
PauliZ(wires=[0]) @ PauliY(wires=[1])
PauliZ(wires=[0]) @ PauliY(wires=[1]) @ PauliZ(wires=[2])
PauliX(wires=[1]) @ PauliX(wires=[2])
PauliX(wires=[1]) @ PauliY(wires=[2])
PauliY(wires=[1]) @ PauliX(wires=[2])
PauliY(wires=[1]) @ PauliY(wires

In [ ]:
from pennylane.pauli import pauli_group
from collections import defaultdict
n_qubits = 3

# size of 64, 4^3
GN = pauli_group(n_qubits)

# Generator set should have size n-k = 2 (Should get generators from S but I know G so I'm generating S from thats)
G1 = qml.Identity(0) @ qml.PauliZ(1) @ qml.PauliZ(2)
G2 = qml.PauliZ(0) @ qml.PauliZ(1) @ qml.Identity(2)

G1 = G1.matrix()
G2 = G2.matrix()

# S size of 4 (Assuming generators that we chose should be able to make up the set of S)
I = qml.Identity(0) @ qml.Identity(1) @ qml.Identity(2)
I = I.matrix()
S = [G1, G2, G1 @ G2, I]
def find_unique_eigenstates(stabilizers):
    eigenvectors = defaultdict(set)
    for i,stabilizer in enumerate(stabilizers):
        eigvals, eigvecs = np.linalg.eig(stabilizer)
        for j in range(len(eigvals)):
            if np.isclose(eigvals[j], 1):
                eigenvectors[i].add(tuple(eigvecs[:,j]))

    sets_of_arrays = list(eigenvectors.values())
    common_arrays = set.intersection(*sets_of_arrays)
    return common_arrays

a = find_unique_eigenstates(S)
print(a)


def calculate_P_with_G():
    I = qml.Identity(0) @ qml.Identity(1) @ qml.Identity(2)
    I = I.matrix()

    # G1 and G2 are diagonal matrix so the eigenvalues are just the values of the diagonal
    # Furthermore, the values are only +/- 1
    # If we add the identity matrix then the eigenvalues become 0 and 2
    # If we divide by 2 all eigenvalues become only 0 or 1
    # why is this important? idk
    p1 = (I + G1)/2
    p2 = (I + G2)/2
    P = p1 @ p2
    return P

def calculate_P_with_S():
    P = np.zeros(G1.shape)
    for elem in S:
        P += elem
    # Same reasoning above, have to divide to get eigenvalue back to 0 or 1 
    P *= 1/(len(S))
    return P
        
P1 = calculate_P_with_G()
P2 = calculate_P_with_S()
print(np.allclose(P1,P2))






In [ ]:
dev = qml.device("default.qubit", wires=3)

@qml.qnode(dev)
def asdf():
    qml.Hadamard(wires=0)
    qml.CNOT(wires=[0, 1])
    qml.PauliZ(wires=0)
    qml.PauliZ(wires=1)
    return qml.state()

S = [G1, G2, G1 @ G2, I]

result = asdf()
print(result)

In [ ]:
dev = qml.device("default.mixed", wires=5,shots=1)


#n = 3
#k = 1



def generate_density_matrix():

    # this is the hadamard shit
    c1 = 1/np.sqrt(2)
    c2 = 1/np.sqrt(2)
    state = np.array([c1, 0, 0, 0, 0, 0, 0, c2])
    #state = np.array([0.5, 0, 0.5, 0, 0.5, 0, 0, 0.5])
    # generate the density matrix
    rho = np.outer(state, np.conj(state).T)
    return rho


@qml.qnode(dev)
def syndrome_measurement(estimation_wires,target_wires):
    # rho = np.zeros((2 ** len(target_wires), 2 ** len(target_wires)), dtype=np.complex128)
    # rho[0,0] = 1
    density_matrix = generate_density_matrix()
    print(density_matrix)
    # should be a density matrix
    qml.QubitDensityMatrix(density_matrix, wires=target_wires)
    
    for wire in estimation_wires:
        qml.Hadamard(wires=wire)
        
    #qml.PauliX(target_wires[0])
    qml.BitFlip(0.75,target_wires[0])
    #qml.PhaseFlip(0.4,target_wires[1])
    #qml.PhaseFlip(0.8,target_wires[2])
    #qml.PauliZ(wires=target_wires[1])

    #Detects if bit 0 or bit 1 flipped
    g1_kron = qml.PauliZ(target_wires[0]) @ qml.PauliZ(target_wires[1]) @ qml.Identity(wires=target_wires[2])

    # Detect if bit 1 or bit 2 flipped
    g2_kron = qml.Identity(wires=target_wires[0]) @ qml.PauliZ(target_wires[1]) @ qml.PauliZ(target_wires[2])
    
    g1_kron = g1_kron.matrix()
    g2_kron = g2_kron.matrix()
    
    #qml.ControlledQubitUnitary(g1_kron, control_wires=estimation_wires[0], wires=target_wires)
    #qml.ControlledQubitUnitary(g2_kron, control_wires=estimation_wires[1], wires=target_wires)

    return qml.density_matrix(target_wires)
    #return [qml.expval(qml.PauliX(estimation_wires[0])),qml.expval(qml.PauliX(estimation_wires[1]))]#

    
    #return qml.sample(wires=estimation_wires)

estimation_wires = [0, 1]
target_wires = [2,3,4]


#print(qml.draw(syndrome_measurement)(estimation_wires,target_wires))
#ans = hadamard_test(estimation_wires,target_wires)
# first = 0
# second = 0
# for i in range(100):
#     a,b = syndrome_measurement(estimation_wires,target_wires)
#     first += a
#     second += b
# print(first)
# print(second)
def density_matrix_to_state(rho):
    # Compute the eigenvectors and eigenvalues of the density matrix
    eigvals, eigvecs = np.linalg.eig(rho)
    # Take the square root of the eigenvalues
    sqrt_eigvals = np.sqrt(eigvals)
    # Multiply each eigenvector by the square root of the corresponding eigenvalue
    state_vectors = [np.multiply(sqrt_eigvals[i], eigvecs[:, i]) for i in range(len(eigvals))]
    # Add up all the resulting vectors to get the state vector
    state_vector = sum(state_vectors)
    # Normalize the state vector
    state_vector /= np.linalg.norm(state_vector)
    return state_vector


rho = syndrome_measurement(estimation_wires,target_wires)
print(rho)

st = density_matrix_to_state(rho)
print(st)



#Detects if bit 0 or bit 1 flipped
g1_kron = qml.PauliZ(0) @ qml.PauliZ(1) @ qml.Identity(2)
g1_kron = g1_kron.matrix()

# Detect if bit 1 or bit 2 flipped
g2_kron = qml.Identity(0) @ qml.PauliZ(1) @ qml.PauliZ(2)
g2_kron = g2_kron.matrix()

I = qml.Identity(0) @ qml.Identity(1) @ qml.Identity(2)
I = I.matrix()
p1 = 0.5 * (I + g1_kron)
p2 = 0.5 * (I + g2_kron)

P = p1 @ p2

num = P @ rho @ P 
# Expectation value of P 
denom = np.trace(rho @ P)
print("trace")
print(denom)
Pdet = num/denom
print(Pdet)
print(np.allclose(rho,Pdet))

#p = create_entangled_state([0])
#num = np.matmul(np.matmul(P,p),P)
# den = np.sum(np.matmul(p,P))
# pDet = num/den
# print(p)
# print(pDet)

In [ ]:
dev = qml.device("default.mixed", wires=5)


#n = 3
#k = 1



def generate_density_matrix():

    # this is the hadamard shit
    c1 = 1/np.sqrt(2)
    c2 = 1/np.sqrt(2)
    state = np.array([c1, 0, 0, 0, 0, 0, 0, c2])
    
    # generate the density matrix
    rho = np.outer(state, np.conj(state).T)
    return rho


@qml.qnode(dev)
def syndrome_measurement(estimation_wires,target_wires):
    # rho = np.zeros((2 ** len(target_wires), 2 ** len(target_wires)), dtype=np.complex128)
    # rho[0,0] = 1
    density_matrix = generate_density_matrix()
    print(density_matrix)
    # should be a density matrix
    qml.QubitDensityMatrix(density_matrix, wires=target_wires)
    
    for wire in estimation_wires:
        qml.Hadamard(wires=wire)

    qml.BitFlip(0.5,target_wires[0])
    #qml.PhaseFlip(0.4,target_wires[1])
    #qml.PhaseFlip(0.8,target_wires[2])
    #qml.PauliZ(wires=target_wires[1])

    #Detects if bit 0 or bit 1 flipped
    g1_kron = qml.PauliZ(target_wires[0]) @ qml.PauliZ(target_wires[1]) @ qml.Identity(wires=target_wires[2])

    # Detect if bit 1 or bit 2 flipped
    g2_kron = qml.Identity(wires=target_wires[0]) @ qml.PauliZ(target_wires[1]) @ qml.PauliZ(target_wires[2])
    
    g1_kron = g1_kron.matrix()
    g2_kron = g2_kron.matrix()
    
    qml.ControlledQubitUnitary(g1_kron, control_wires=estimation_wires[0], wires=target_wires)
    qml.ControlledQubitUnitary(g2_kron, control_wires=estimation_wires[1], wires=target_wires)
    
    return qml.density_matrix(target_wires)
    #return [qml.expval(qml.PauliX(estimation_wires[0])),qml.expval(qml.PauliX(estimation_wires[1]))]#

    
    #return qml.sample(wires=estimation_wires)

estimation_wires = [0, 1]
target_wires = [2,3,4]


#print(qml.draw(syndrome_measurement)(estimation_wires,target_wires))
#ans = hadamard_test(estimation_wires,target_wires)
# first = 0
# second = 0
# for i in range(100):
#     a,b = syndrome_measurement(estimation_wires,target_wires)
#     first += a
#     second += b
# print(first)
# print(second)

rho = syndrome_measurement(estimation_wires,target_wires)
print(rho)
#Detects if bit 0 or bit 1 flipped
g1_kron = qml.PauliZ(0) @ qml.PauliZ(1) @ qml.Identity(2)
g1_kron = g1_kron.matrix()

# Detect if bit 1 or bit 2 flipped
g2_kron = qml.Identity(0) @ qml.PauliZ(1) @ qml.PauliZ(2)
g2_kron = g2_kron.matrix()

I = qml.Identity(0) @ qml.Identity(1) @ qml.Identity(2)
I = I.matrix()
p1 = 0.5 * (I + g1_kron)
p2 = 0.5 * (I + g2_kron)

P = p1 @ p2
print(P)
print("no way")
print(np.allclose(P,P.T))
num = P @ rho @ P 
# Expectation value of P 
denom = np.trace(rho @ P)
print(denom)
Pdet = num/denom
print(Pdet)
print(np.allclose(rho,Pdet))

#p = create_entangled_state([0])
#num = np.matmul(np.matmul(P,p),P)
# den = np.sum(np.matmul(p,P))
# pDet = num/den
# print(p)
# print(pDet)

In [ ]:
def generate_density_matrix():

    # this is the hadamard shit
    c1 = 1/np.sqrt(2)
    c2 = 1/np.sqrt(2)
    state = np.array([c1, 0, 0, 0, 0, 0, 0, c2])
    
    # generate the density matrix
    rho = np.outer(state, np.conj(state).T)
    return rho

U1 = qml.PauliZ(0) @ qml.Identity(1) @ qml.Identity(2)
U1 = U1.matrix()
U2 = qml.Identity(0) @ qml.PauliZ(1) @ qml.Identity(2)
U2 = U2.matrix()
U3 = qml.Identity(0) @ qml.Identity(1) @ qml.PauliZ(2)
U3 = U3.matrix()

Ulist = [U1, U2, U3]

p = generate_density_matrix()
pid = U1 @ p @ U1.conj().T
pid = U2 @ pid @ U2.conj().T
pid = U3 @ pid @ U3.conj().T
print("pid")
print(pid)

dev = qml.device("default.mixed", wires=3)
@qml.qnode(dev)
def add_noise(wires):
    density_matrix = generate_density_matrix()
    qml.QubitDensityMatrix(density_matrix, wires)
    for i,U in enumerate(Ulist):
        qml.QubitUnitary(U,wires)
        qml.BitFlip(0.5,wires[i])
        qml.QubitUnitary(P,wires)

    return qml.density_matrix(wires)

wires = [0, 1,2]

rho_det_prime = add_noise(wires)
print("rho_det_prime")
print(rho_det_prime)
rho_det = rho_det_prime/np.trace(rho_det_prime)
print(rho_det)
print(np.allclose(rho_det,pid))

In [26]:
# FIG 2
# U list already has the noise in it
# U_list[i] = E(.) U(.)

# Usage: fig_2_circ = generate_fig_2([U1, U2, U3], qml.PauliX())
#        a_list, o_meas = fig_2_circ(rho_naught, <uniform sample of S1 of length L>, <ancilla bits>, <target_bits>)
#        a_s = multiply_all(a_list)
#        b_s = a_s * o_meas


def generate_fig_2(U_list, O):
    n_estimation_wires = len(U_list)
    n_target_wires = 3

    dev = qml.device("default.mixed", wires=(n_estimation_wires + n_target_wires))

    @qml.qnode(dev)
    def VQED(rho_naught, stabilizers_list, estimation_wires, target_wires):
        a_list = []
        qml.QubitDensityMatrix(rho_naught, wires=target_wires)
        for wire in estimation_wires:
            qml.Hadamard(wires=wire)

        for l, (stabilizer_i_l, stabilizer_j_l) in enumerate(stabilizers_list):
            U = U_list[l]
            qml.QubitUnitary(U, wires=target_wires)
            qml.ControlledQubitUnitary(stabilizer_i_l, control_wires=estimation_wires[l], wires=target_wires, control_values="0")
            qml.ControlledQubitUnitary(stabilizer_j_l, control_wires=estimation_wires[l], wires=target_wires, control_values="1")

        for wire in estimation_wires:
            a_list.append(qml.expval(qml.PauliX(wire)))
        
        # for wire in target_wires:
        #     o_list.append(qml.expval(qml.PauliX(wire)))

        o = qml.expval(O)
        print(o)
        return a_list + [o]
    return VQED


def generate_density_matrix():

    # this is the hadamard shit
    c1 = 1/np.sqrt(2)
    c2 = 1/np.sqrt(2)
    state = np.array([c1, 0, 0, 0, 0, 0, 0, c2])
    
    # generate the density matrix
    rho = np.outer(state, np.conj(state).T)
    return rho

U1 = qml.PauliZ(0) @ qml.Identity(1) @ qml.Identity(2)
U1 = U1.matrix()
U2 = qml.Identity(0) @ qml.PauliZ(1) @ qml.Identity(2)
U2 = U2.matrix()
U3 = qml.Identity(0) @ qml.Identity(1) @ qml.PauliZ(2)
U3 = U3.matrix()

Ulist = [U1, U2, U3]

p = generate_density_matrix()
pid = U1 @ p @ U1.conj().T
pid = U2 @ pid @ U2.conj().T
pid = U3 @ pid @ U3.conj().T

f = generate_fig_2(Ulist,qml.PauliX(3) @ qml.PauliX(4))

G1 = qml.Identity(0) @ qml.PauliZ(1) @ qml.PauliZ(2)
G2 = qml.PauliZ(0) @ qml.PauliZ(1) @ qml.Identity(2)

G1 = G1.matrix()
G2 = G2.matrix()

# S size of 4 (Assuming generators that we chose should be able to make up the set of S)
I = qml.Identity(0) @ qml.Identity(1) @ qml.Identity(2)
I = I.matrix()
S = [G1, G2, G1 @ G2, I]
# L = 3

uniform_sample = [[S[0],S[1]],[S[2],S[3]],[S[0],S[1]]]

estimation_wires=[0,1,2]
target_wires=[3,4,5]


f(generate_density_matrix(),uniform_sample,estimation_wires,target_wires)

    

expval(PauliX(wires=[3]) @ PauliX(wires=[4]))


array([1., 1., 1., 0.])

In [111]:
# FIG 2
# U list already has the noise in it
# U_list[i] = E(.) U(.)

# Usage: fig_2_circ = generate_fig_2([U1, U2, U3], qml.PauliX())
#        a_list, o_meas = fig_2_circ(rho_naught, <uniform sample of S1 of length L>, <ancilla bits>, <target_bits>)
#        a_s = multiply_all(a_list)
#        b_s = a_s * o_meas


def generate_fig_2(U_list, O):
    n_estimation_wires = len(U_list)
    n_target_wires = 3

    dev = qml.device("default.mixed", wires=(n_estimation_wires + n_target_wires))

    @qml.qnode(dev)
    def VQED(rho_naught, stabilizers_list, estimation_wires, target_wires):
        measurements = []
        qml.QubitDensityMatrix(rho_naught, wires=target_wires)
        for wire in estimation_wires:
            qml.Hadamard(wires=wire)


        for l, (stabilizer_i_l, stabilizer_j_l) in enumerate(stabilizers_list):
            U = U_list[l]
            qml.QubitUnitary(U, wires=target_wires)
            #qml.BitFlip(0.5,wires=target_wires[1])
            qml.ControlledQubitUnitary(stabilizer_i_l, control_wires=estimation_wires[l], wires=target_wires, control_values="0")
            qml.ControlledQubitUnitary(stabilizer_j_l, control_wires=estimation_wires[l], wires=target_wires, control_values="1")
        for wire in estimation_wires:
            measurements.append(qml.expval(qml.PauliX(wire)))

        
        # for wire in target_wires:
        #     o_list.append(qml.expval(qml.PauliX(wire)))

        measurements.append(qml.expval(O))
        return measurements
    return VQED


def generate_density_matrix():

    # this is the hadamard shit
    c1 = 1/np.sqrt(2)
    c2 = 1/np.sqrt(2)
    state = np.array([0.5, 0, 0.5, 0, 0, 0.5, 0, 0.5])
    
    # generate the density matrix
    rho = np.outer(state, np.conj(state).T)
    return rho

U1 = qml.PauliZ(0) @ qml.Identity(1) @ qml.Identity(2)
U1 = U1.matrix()
U2 = qml.Identity(0) @ qml.PauliZ(1) @ qml.Identity(2)
U2 = U2.matrix()
U3 = qml.Identity(0) @ qml.Identity(1) @ qml.PauliZ(2)
U3 = U3.matrix()
U4 = qml.PauliZ(0) @ qml.PauliZ(1) @ qml.Identity(2)
U4 = U4.matrix()

Ulist = [U1, U2, U3, U4]

pid = generate_density_matrix()
pid = U1 @ pid @ U1.conj().T
pid = U2 @ pid @ U2.conj().T
pid = U3 @ pid @ U3.conj().T
pid = U4 @ pid @ U4.conj().T

Observable = qml.PauliX(4) @ qml.Hadamard(5) @ qml.PauliZ(6)
f = generate_fig_2(Ulist,Observable)

G1 = qml.Identity(0) @ qml.PauliZ(1) @ qml.PauliZ(2)
G2 = qml.PauliZ(0) @ qml.PauliZ(1) @ qml.Identity(2)

G1 = G1.matrix()
G2 = G2.matrix()

# S size of 4 (Assuming generators that we chose should be able to make up the set of S)
I = qml.Identity(0) @ qml.Identity(1) @ qml.Identity(2)
I = I.matrix()
S = [G1, G2, G1 @ G2, I]
# L = 4

uniform_sample = [random.sample(S,2),random.sample(S,2),random.sample(S,2),random.sample(S,2)]
estimation_wires=[0,1,2,3]
target_wires=[4,5,6]

a = 0
b = 0
N = 5


for i in range(N):
    uniform_sample = [random.sample(S,2),random.sample(S,2),random.sample(S,2),random.sample(S,2)]

    measurements = f(generate_density_matrix(),uniform_sample,estimation_wires,target_wires)
    print(measurements)
    a_s = measurements[0]
    for elem in range(1,len(measurements)-1):
        a_s *= measurements[elem]
    print(a_s)
    b_s = a_s * measurements[-1]
    a += a_s
    b += b_s

a *= 1/N
b *= 1/N
print(a)
print(b)
print(b/a)



[0. 0. 0. 0. 0.]
0.0
[0. 1. 1. 0. 0.]
0.0
[0. 1. 0. 1. 0.]
0.0
[0. 0. 0. 1. 0.]
0.0
[0. 1. 1. 0. 0.]
0.0
0.0
0.0
nan


C:\Users\marco\AppData\Local\Temp\ipykernel_5048\2886881370.py:110: RuntimeWarning: invalid value encountered in double_scalars
  print(b/a)


In [11]:
# symmetry expansion

def generate_density_matrix():

    # this is the hadamard shit
    c1 = 1/np.sqrt(2)
    c2 = 1/np.sqrt(2)
    state = np.array([0.5, 0.5, 0.5, 0,0,0,0.5,0])
    
    # generate the density matrix
    rho = np.outer(state, np.conj(state).T)
    return rho

dev = qml.device("default.mixed", wires=3)
@qml.qnode(dev)
def generate_noise(wires):
    density_matrix = generate_density_matrix()
    qml.QubitDensityMatrix(density_matrix, wires=wires)
    qml.BitFlip(0.75,wires[0])
    qml.BitFlip(0.35,wires[1])
    qml.BitFlip(1,wires[2])
    return qml.density_matrix(wires)

# Assume rho is equal to rho det because the state is in the codespace
rho_det = generate_density_matrix()

O = qml.Identity(0) @ qml.Identity(1) @ qml.PauliZ(2)
O = O.matrix()
rho =  generate_noise([0,1,2])
g1_kron = qml.PauliZ(0) @ qml.PauliZ(1) @ qml.Identity(2)
g1_kron = g1_kron.matrix()

# Detect if bit 1 or bit 2 flipped
g2_kron = qml.Identity(0) @ qml.PauliZ(1) @ qml.PauliZ(2)
g2_kron = g2_kron.matrix()

I = qml.Identity(0) @ qml.Identity(1) @ qml.Identity(2)
I = I.matrix()
p1 = 0.5 * (I + g1_kron)
p2 = 0.5 * (I + g2_kron)

P = p1 @ p2

ans = np.trace(rho_det @ O)
print(rho_det @ O)
print(ans)


num = np.trace(rho @ O @ P)
denom = np.trace(rho @ P)
print(num)
print(denom)
print(num/denom)

N = 500
G1 = qml.Identity(0) @ qml.PauliZ(1) @ qml.PauliZ(2)
G2 = qml.PauliZ(0) @ qml.PauliZ(1) @ qml.Identity(2)

G1 = G1.matrix()
G2 = G2.matrix()
S = [G1, G2, G1 @ G2, I]
a = 0
b = 0

def matrices_commute(A, B):
    commutator = np.dot(A, B) - np.dot(B, A)
    return np.allclose(commutator, np.zeros_like(commutator))

print(matrices_commute(O,P))
for i in range(N):
    Si = random.sample(S,1)[0]
    a_s = np.trace(rho @ Si)
    b_s = np.trace(rho @ O @ Si)
    a += a_s
    b += b_s


print(b/a)



[[ 0.25 -0.25  0.25  0.    0.    0.    0.25  0.  ]
 [ 0.25 -0.25  0.25  0.    0.    0.    0.25  0.  ]
 [ 0.25 -0.25  0.25  0.    0.    0.    0.25  0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.25 -0.25  0.25  0.    0.    0.    0.25  0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.  ]]
0.5
(-0.1875000000000074+0j)
(0.26875000000001525+0j)
(-0.6976744186046392+0j)
True
(-0.7191568505889535+0j)


In [60]:
#Detects if bit 0 or bit 1 flipped
g1_kron = qml.PauliZ(0) @ qml.PauliZ(1) @ qml.Identity(2)
g1_kron = g1_kron.matrix()

# Detect if bit 1 or bit 2 flipped
g2_kron = qml.Identity(0) @ qml.PauliZ(1) @ qml.PauliZ(2)
g2_kron = g2_kron.matrix()

I = qml.Identity(0) @ qml.Identity(1) @ qml.Identity(2)
I = I.matrix()
p1 = 0.5 * (I + g1_kron)
p2 = 0.5 * (I + g2_kron)

P = p1 @ p2 

rho = np.zeros((2 ** len(target_wires), 2 ** len(target_wires)), dtype=np.complex128)
rho[0,0] = 1
# should be a density matrix
print(rho)

print(np.allclose(P,(P @ P)))

num = P @ rho @ P 
denom = np.trace(rho @ P)

num/denom

[[1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]]
True


array([[1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]])

In [121]:
print(qml.version())

0.28.0


In [47]:
dev = qml.device("default.mixed", wires=3, shots=1)

estimation_wires = [0, 1, 2]
target_wires = [3]

@qml.qnode(dev)
def hadamard_test(wires):
    for wire in estimation_wires:
        qml.Hadamard(wires=wire)
    
    qml.PauliX(wires=[1])
    qml.PauliX(wires=[1])
    qml.PauliY(wires=[2])
    qml.PauliY(wires=[2])
    
    #qml.PauliZ(wires=wires[1])
    #qml.PauliZ(wires=wires[2])
    qml.Hadamard(wires=wires[0])
    #qml.PhaseFlip(1,wires=[0])
    #create_entangled_state(wires)
    #qml.Hadamard(wires=wires[2])
    qml.ctrl(qml.PauliZ,control=wires[0])(wires=[1])
    qml.ctrl(qml.PauliZ,control=wires[0])(wires=[2])
    return qml.expval(qml.PauliX(0) @ qml.PauliX(1))
    return qml.expval(qml.PauliX(0)), qml.expval(qml.PauliX(1)), qml.expval(qml.PauliX(2))
    #return qml.expval(qml.PauliX(0))
    #return qml.measure(qml.PauliX(0))
    #return qml.sample(qml.PauliZ(wires=0))
    #return qml.measure(qml.PauliX(0))
    #return qml.measure(qml.PauliX(0))
print(qml.draw(hadamard_test)([0,1,2]))

#create_entangled_state(wires=[0,1,2])

# sum = 0
# sum2 = 0
# #sum3 = 0
# for i in range(0,1000):
#     a,b = hadamard_test(wires=[0,1])
#     sum += a
#     sum2 += b
#     #sum3 += c
# print(sum)
# print(sum2)
# #print(sum3)
hadamard_test(wires=[0,1,2])

#p = create_entangled_state([0])
#num = np.matmul(np.matmul(P,p),P)
# den = np.sum(np.matmul(p,P))
# pDet = num/den
# print(p)
# print(pDet)


0: ──H────╭●─╭●─┤ ╭<X@X>
1: ──X──X─╰Z─│──┤ ╰<X@X>
2: ──Y──Y────╰Z─┤       


tensor(-1., requires_grad=True)

In [83]:
dev = qml.device("default.qubit", wires=2)

@qml.qnode(dev)
def asdf():
    qml.Hadamard(wires=0)
    qml.CNOT(wires=[0, 1])
    qml.PauliX(wires=0)
    qml.PauliX(wires=1)
    return qml.state()

result = asdf()
print(result)

[0.70710678+0.j 0.        +0.j 0.        +0.j 0.70710678+0.j]


In [84]:
@qml.qnode(qml.device('lightning.qubit', wires=2))
def circuit():
    qml.Hadamard(0)
    qml.CRX(5, wires=[0, 1])
    qml.Rot([1,2,3], wires=[1])
    qml.CRX(7, wires=[0, 1])
    return qml.expval(qml.PauliZ(0) @ qml.PauliZ(1))

In [494]:
print(qml.draw(circuit))

<function circuit at 0x0000021B287BB160>


In [538]:
import pennylane as qml

n_qubits = 2  # number of physical qubits
k_logical = 1  # number of logical qubits
G1 = [qml.PauliX(0) @ qml.PauliX(1)]  # stabilizer generator

dev = qml.device('default.qubit', wires=n_qubits)

@qml.qnode(dev)
def test():
    # apply Hadamard gates to all qubits
    for i in range(n_qubits):
        qml.Hadamard(wires=i)

    # measure stabilizer generator G1
    for i in range(k_logical):
        qml.PauliZ(wires=range(n_qubits)) @ qml.Hadamard(wires=n_qubits+i)

    # measure in the computational basis
    return [qml.sample(qml.PauliZ(i)) for i in range(n_qubits)]

# run the circuit and print the measurement results
measurement_results = syndrome_measurement_circuit()
print(measurement_results)

ValueError: PauliZ: wrong number of wires. 2 wires given, 1 expected.

In [19]:
# symmetry expansion

def generate_density_matrix():

    # this is the hadamard shit
    c1 = 1/np.sqrt(2)
    c2 = 1/np.sqrt(2)
    state = np.array([0.5, 0.5, 0.5, 0,0,0,0.5,0])
    
    # generate the density matrix
    rho = np.outer(state, np.conj(state).T)
    return rho

dev = qml.device("default.mixed", wires=3)
@qml.qnode(dev)
def generate_noise(wires):
    density_matrix = generate_density_matrix()
    qml.QubitDensityMatrix(density_matrix, wires=wires)
    qml.BitFlip(0.75,wires[0])
    qml.BitFlip(0.35,wires[1])
    qml.BitFlip(1,wires[2])
    return qml.density_matrix(wires)

# Assume rho is equal to rho det because the state is in the codespace
rho_det = generate_density_matrix()

O = qml.Identity(0) @ qml.Identity(1) @ qml.PauliZ(2)
O = O.matrix()
rho =  generate_noise([0,1,2])
g1_kron = qml.PauliZ(0) @ qml.PauliZ(1) @ qml.Identity(2)
g1_kron = g1_kron.matrix()

# Detect if bit 1 or bit 2 flipped
g2_kron = qml.Identity(0) @ qml.PauliZ(1) @ qml.PauliZ(2)
g2_kron = g2_kron.matrix()

I = qml.Identity(0) @ qml.Identity(1) @ qml.Identity(2)
I = I.matrix()
p1 = 0.5 * (I + g1_kron)
p2 = 0.5 * (I + g2_kron)

P = p1 @ p2

ans = np.trace(rho_det @ O)
print(rho_det @ O)
print(ans)


num = np.trace(rho @ O @ P)
denom = np.trace(rho @ P)
print(num)
print(denom)
print(num/denom)

N = 500
G1 = qml.Identity(0) @ qml.PauliZ(1) @ qml.PauliZ(2)
G2 = qml.PauliZ(0) @ qml.PauliZ(1) @ qml.Identity(2)

G1 = G1.matrix()
G2 = G2.matrix()
S = [G1, G2, G1 @ G2, I]
a = 0
b = 0

def matrices_commute(A, B):
    commutator = np.dot(A, B) - np.dot(B, A)
    return np.allclose(commutator, np.zeros_like(commutator))

print(matrices_commute(O,P))
for i in range(N):
    Si = random.sample(S,1)[0]
    a_s = np.trace(rho @ Si)
    b_s = np.trace(rho @ O @ Si)
    a += a_s
    b += b_s


print(b/a)



[[ 0.25 -0.25  0.25  0.    0.    0.    0.25  0.  ]
 [ 0.25 -0.25  0.25  0.    0.    0.    0.25  0.  ]
 [ 0.25 -0.25  0.25  0.    0.    0.    0.25  0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.25 -0.25  0.25  0.    0.    0.    0.25  0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.  ]]
0.5
(-0.1875000000000074+0j)
(0.26875000000001525+0j)
(-0.6976744186046392+0j)
True
(-0.696177847113873+0j)


In [16]:


# FIG 2
# U list already has the noise in it
# U_list[i] = E(.) U(.)

# Usage: fig_2_circ = generate_fig_2([U1, U2, U3], qml.PauliX())
#        a_list, o_meas = fig_2_circ(rho_naught, <uniform sample of S1 of length L>, <ancilla bits>, <target_bits>)
#        a_s = multiply_all(a_list)
#        b_s = a_s * o_meas


def generate_fig_2(U_list, O):
    n_estimation_wires = 1
    n_target_wires = 3

    dev = qml.device("default.mixed", wires=(n_estimation_wires + n_target_wires))

    @qml.qnode(dev)
    def VQED(rho_naught, stabilizers_list, estimation_wire, target_wires):
        measurements = []
        qml.QubitDensityMatrix(rho_naught, wires=target_wires)
        qml.Hadamard(wires=estimation_wire)

        for l, (stabilizer_i_l, stabilizer_j_l) in enumerate(stabilizers_list):
            U = U_list[l]
            qml.QubitUnitary(U, wires=target_wires)
            qml.ControlledQubitUnitary(stabilizer_i_l, control_wires=estimation_wire, wires=target_wires, control_values="0")
            qml.ControlledQubitUnitary(stabilizer_j_l, control_wires=estimation_wire, wires=target_wires, control_values="1")

        # for wire in estimation_wire:
        measurements.append(qml.expval(qml.PauliX(estimation_wire)))
        measurements.append(qml.expval(qml.Hermitian(O, wires=target_wires)))
        return measurements

    return VQED

U1 = qml.PauliZ(0) @ qml.Identity(1) @ qml.Identity(2)
U1 = U1.matrix()
U2 = qml.Identity(0) @ qml.PauliZ(1) @ qml.Identity(2)
U2 = U2.matrix()
U3 = qml.Identity(0) @ qml.Identity(1) @ qml.PauliZ(2)
U3 = U3.matrix()
U4 = qml.PauliZ(0) @ qml.PauliZ(1) @ qml.Identity(2)
U4 = U4.matrix()

Ulist = [U1, U2, U3, U4]

pid = generate_density_matrix()
pid = U1 @ pid @ U1.conj().T
pid = U2 @ pid @ U2.conj().T
pid = U3 @ pid @ U3.conj().T
pid = U4 @ pid @ U4.conj().T

Observable = qml.PauliX(4) @ qml.Hadamard(5) @ qml.PauliZ(6)
f = generate_fig_2(Ulist,qml.matrix(Observable))

G1 = qml.Identity(0) @ qml.PauliZ(1) @ qml.PauliZ(2)
G2 = qml.PauliZ(0) @ qml.PauliZ(1) @ qml.Identity(2)

G1 = G1.matrix()
G2 = G2.matrix()

# S size of 4 (Assuming generators that we chose should be able to make up the set of S)
I = qml.Identity(0) @ qml.Identity(1) @ qml.Identity(2)
I = I.matrix()
S = [G1, G2, G1 @ G2, I]
# L = 4

uniform_sample = [random.sample(S,2),random.sample(S,2),random.sample(S,2),random.sample(S,2)]
estimation_wire=0
target_wires=[1,2,3]

a = 0
b = 0
N = 100


for i in range(N):
    uniform_sample = [random.sample(S,2),random.sample(S,2),random.sample(S,2),random.sample(S,2)]
    measurements = f(generate_density_matrix(),uniform_sample,estimation_wire,target_wires)
    a_s = measurements[0]
    b_s = a_s * measurements[-1]
    a += a_s
    b += b_s

# L = 4
a *= 1/N
b *= 1/N
print(a)
print(b)
print(b/a)

0.15999999999999998
0.017677669529663664
0.11048543456039792
